In [1]:
from adversarial import *

In [2]:
vae = VAE.load("trained_models/VAE")
cnn = load_model("trained_models/CNN/classifier.h5")
xai = xai_model(vae.decoder, cnn, input_shape=(12,))

# input images
# samples_test, sample_labels_test = load_samples_for_test_folder(img_dir='./contrib/DLFuzz/MNIST/seeds_50')
samples_test, sample_labels_test = load_samples_for_test(200)

# prepare
samples_view = samples_test
sample_labels_view = sample_labels_test
# samples_view = samples
# sample_labels_view = sample_labels

x_view = np.reshape(samples_view, (-1, 784))
y_onehot_view = tf.one_hot(tf.constant(sample_labels_view), depth=10).numpy()

K.set_learning_phase(0)
# dlfuzz = DLFuzz(cnn)
clhans = CleverHans(cnn)

2024-06-07 10:48:08.731412: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


/opt/anaconda3/envs/bachelor/lib/python3.9/site-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [3]:
for i in tqdm(range(len(x_view))):
    # get original image
    image_org = samples_view[i]
    label_org = sample_labels_view[i]

    # generate dlfuzz image
    # image_adv = dlfuzz.generate_adversarial_image(image_org)  # of shape (28, 28, 1)
    image_adv = clhans.generate_adversarial_image(image_org)  # of shape (28, 28, 1)
    label = np.argmax(cnn.predict(np.array([image_adv]))[0])

    # List of images and their titles
    images = [image_org, image_adv]
    titles = [f'image_org_{label_org}', f'image_adv_{label}']

    # Plot the images
    if label != label_org:
        plot_image_comparison_two(images, titles)

        kl_fuzz = kl_divergence(image_org, image_adv)
        print(f"kl: {kl_fuzz}")

        ws_fuzz = ws_distance(image_org, image_adv)
        print(f"ws: {ws_fuzz}")

  0%|                                                   | 0/200 [00:00<?, ?it/s]


UnparsedFlagAccessError: Trying to access flag --eps before flags were parsed.